In [1]:
#Преквезиты: 
#selenium (pip install selenium)
#pandas (pip install pandas)
#скомпилированный PhantomJS или Firefox
#ЕСЛИ у вас нет монитора, Xvfb (sudo apt-get install xvfb)

#полный-детальный процесс установки сюда:
#pip install selenium
#pip install pandas
#sudo apt-get update
#sudo apt-get install firefox
#sudo apt-get install xvfb
#cd path_to_notebooks
#git clone https://github.com/BladeCarrier/pyzure
#запустить тетрадку


import selenium.webdriver as wd
from wrapper.model_wrapper import model_wrapper as wrapper
import wrapper.configure_browser as configure
import pandas as pd
import time

In [2]:
#Если у Вас нету монитора...
import os
if os.getenv("DISPLAY") is None:
    os.system("Xvfb :10 -ac -screen 0 1600x1200x24+32 &") #то он у Вас есть. Офигенная плазма во всю виртуальную стену
    %env DISPLAY=:10

env: DISPLAY=:10


In [3]:
#split the data
data =pd.read_csv("training.csv")
del data["EventId"]
del data["Weight"]
train = data[:100000]
test = data[100000:]
train.to_csv("train",index = False)
test.to_csv("test",index = False)
del data, train,test


In [4]:
#Скрытый режим
#mauser = configure.phantomjs("phantomjs_win/bin/phantomjs.exe")
#Сюда нужно запихнуть путь к бинарнику Phantom JS
#где его достать: http://phantomjs.org/download.html

#открытый режим на Windows
#mauser = configure.firefox_portable("FirefoxPortable/FirefoxPortable.exe")
#на linux
mauser = configure.firefox_default()
#на MacOS
#mauser = wd.Safari()
#mauser.maximize_window()


In [5]:
#инициализируем враппер
wr = wrapper(mauser)
#логинимся
wr.sign_in("sasha_panin@mail.ru","243414")

signing in...
signed in as sasha_panin@mail.ru


In [6]:
#Загружаем csv-шку в Azure, eсли её там нет
wr.add_dataset_by_path("train",".csv",change_name =True,new_name = "higgs_nice")
#Это работает отлично в Firefox/Safari, но способа запилить её в PhantomJS я ещё не нашёл

adding dataset from /notebooks/pyzure/train
waiting for file to upload
higgs_funny dataset added


In [7]:
#Создаём и обустраиваем типовой эксперимент
wr.create_new_experiment("megaexp")
###                dataset,     y column,    method,                   problem type,   ...more args...
wr.setup_model("higgs_nice","Label","Multiclass Decision Forest","Classification")
#Запускаем эксперимент
wr.run_experiment()
wr.create_scoring_experiment()
wr.run_experiment()

creating new experiment
created experiment with name: megaexp
deploying machinelearning model experiment
dataset higgs_funny deployed
Multiclass Decision Forest deployed
trainer deployed
Label selected as target column
scorer deployed
deployed service Input
deployed service Output
wiring elements
experiment deployment finished
Running...
Creating scoring experiment...
Running...


In [9]:
#делаем из него сервис
url,api_key = wr.publish_experiment()
time.sleep(10) #microsoft servive publishing delay...

Publishing...


In [10]:
print url,api_key

https://ussouthcentral.services.azureml.net/workspaces/d9866811e0d848e8adbcb171aabfcca3/services/171f4d97d0fc4ad183eea1ce09e1e84c/execute?api-version=2.0&details=true 5RIT/bYIsMeOogoyhZVRAvpZcXVibhiMtZQzR8IGVwRWgwh+f7bxjjU+KewcGCCDK8fBA8Pmv4vEQBc4oHGk+g==


In [11]:
test=pd.read_csv("test")[:100]

In [12]:
res= wr.apply_service(test,url,api_key,)

encoding data
sending request
loading response
parsing response


In [13]:
res["Scored Labels"]

0     s
1     b
2     b
3     s
4     b
5     s
6     s
7     b
8     b
9     b
10    b
11    b
12    s
13    b
14    b
...
85    s
86    b
87    b
88    b
89    s
90    b
91    s
92    b
93    b
94    b
95    s
96    b
97    b
98    s
99    s
Name: Scored Labels, Length: 100, dtype: object

In [14]:
res

,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,...,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt,Label,"Scored Probabilities for Class ""b""","Scored Probabilities for Class ""s""",Scored Labels
0,130.925,4.683,88.164,6.587,-999,-999,-999,2.983,6.587,91.476,...,-999,-999,-999,-999,-999,0,s,0.0625,0.9375,s
1,82.182,64.538,64.631,29.972,-999,-999,-999,3.111,29.972,59.667,...,-999,-999,-999,-999,-999,0,b,0.75,0.25,b
2,77.757,51.504,60.471,30.244,-999,-999,-999,2.127,30.244,62.392,...,-999,-999,-999,-999,-999,0,b,1,0,b
3,116.669,35.082,71.553,101.94,-999,-999,-999,1.92,1.294,185.678,...,-2.66,2.773,-999,-999,-999,102.9,s,0.125,0.875,s
4,-999,81.44,47.316,40.477,-999,-999,-999,1.629,28.682,121.627,...,0.59,-2.935,-999,-999,-999,68.223,b,1,0,b
5,109.145,50.047,80.361,29.164,-999,-999,-999,2.961,29.164,74.795,...,-999,-999,-999,-999,-999,0,s,0.25,0.75,s
6,180.122,12.378,94.168,27.424,-999,-999,-999,2.675,27.424,103.742,...,-999,-999,-999,-999,-999,0,s,0.375,0.625,s
7,84.275,40.197,46.432,244.08,1.022,108.466,2.055,0.754,58.212,592.469,...,-1.011,1.374,91.808,-2.033,1.264,455.101,b,1,0,b
8,68.767,10.432,47.128,27.786,-999,-999,-999,2.527,27.786,49.802,...,-999,-999,-999,-999,-999,0,b,0.875,0.125,b
9,-999,174.255,428.132,38.248,-999,-999,-999,4.425,10.406,141.403,...,0.315,-2.983,-999,-999,-999,30.365,b,0.75,0.25,b


In [15]:
from sklearn.metrics import accuracy_score

In [17]:
print accuracy_score(res["Label"],res["Scored Labels"])

0.83


In [35]:
print res['Scored Probabilities for Class "s"']

0     0.9375
1       0.25
2          0
3      0.875
4          0
5       0.75
6      0.625
7          0
8      0.125
9       0.25
10     0.125
11       0.5
12     0.625
13     0.375
14      0.25
...
85    0.875
86    0.375
87      0.5
88    0.125
89    0.625
90     0.25
91    0.625
92        0
93        0
94    0.125
95    0.625
96        0
97     0.25
98    0.625
99        1
Name: Scored Probabilities for Class "s", Length: 100, dtype: object


In [30]:
%time
test2=pd.read_csv("test")

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 20 µs


In [31]:
%time
print test2.shape
out2= wr.apply_service(test2,url,api_key,)["Scored Labels"]

CPU times: user 7 µs, sys: 1 µs, total: 8 µs
Wall time: 14.1 µs
(150000, 31)
encoding data
sending request
loading response
parsing response


Он врёт, прошло секунд 20!

In [32]:
print accuracy_score(test2["Label"],out2)

0.818306666667
